## Importing Libraries

In [34]:
import os
import re
import time
import shlex
from IPython.display import clear_output

## Indexing

In [2]:
tokens = {}
documentID = 0
path = "dataset"

In [3]:
# Loading the Stop Words
with open("stopwords.txt") as f:
    for line in f:
        stopwords = line.split(", ")

In [4]:
# Read all the tokens
for root, dirs, files in os.walk(path):
    for file in files:
        with open(os.path.join(path, file)) as f:
                documentID += 1
                line_tokens = []
                for line in f:
                    line_tokens = line.split()
                    for each in line_tokens:
                        if each not in stopwords:
                            if each not in tokens:
                                tokens[each] = [documentID]
                            else:
                                tokens[each].append(documentID)

In [5]:
tokens

{'data': [1, 2, 3],
 'new': [1],
 'oil': [1],
 'linkedin': [2],
 'predicts': [2],
 'mining': [2],
 'statistics': [2],
 'bubble': [2],
 'burst': [2],
 'big': [3],
 'future': [3],
 'arpan': [3]}

In [6]:
# Create Inverted Index
with open("index/invertedindex.txt", "w") as f:
    for key in sorted(tokens):
        f.write(key)
        f.write(" ")
        value = ",".join(str(v) for v in tokens[key])
        f.write(value)
        f.write("\n")

In [7]:
#Rotate each word to create the permuterm index
def rotate(str, n):
    return str[n:] + str[:n]

# Create Permuterm Index
with open("index/permutermindex.txt","w") as f:
    keys = tokens.keys()
    for key in sorted(keys):
        dkey = key + "$"
        for i in range(len(dkey),0,-1):
            out = rotate(dkey,i)
            f.write(out)
            f.write(" ")
            f.write(key)
            f.write("\n")

## Querying

In [24]:
#This is a function used for matching the excat queries and not wildcard (eg: 'data', 'future' etc.)
def excat_match(term):
    output_list=[]
    #This is the list contating document IDs having the required word in query
    docID = []
    docID.append(inverted[term])

    temp = []
    for x in docID:
        for y in x:
            temp.append(y)     

    temp = [int(x) for x in temp]
    documentID = 0
    path = "dataset"
    for root, dirs, files in os.walk(path):
        for file in files:
            documentID = documentID + 1
            with open(os.path.join(path, file)) as f:
                for text in f:
                    if documentID in temp:
                        #print(file + "\n" + text + "\n")
                        output_list.append(file + '\n'+  text)
            f.close()
    return(output_list)

In [11]:
#This function will be used for Wildcard Query Processing in case query has more than 1 wildcards (eg: f*t*r etc.)
def bitwise_and(A,B):
    return set(A).intersection(B)

In [ ]:
#This is the primary function which will be used for querying purpose. It handles all sorts of Wild card queries, namely,
#1) X* --> $X 
#2) *X --> X$*
#3) X*Y --> Y$X*
#4) X*Y*Z --> (Z$X*) and (Y*)
#5) *X* --> can be converted to X* form

In [17]:
def querying(query):
    output_list=[]
    final_result=[]

    # Split query and determine it's type
    parts = query.split("*")
    
    #print(parts)

    final_result.append('These are the sub parts of the query')
    final_result.append(parts)
    
    #These are the different cases formed depending on the number of wild card characters and their position in our query
    if len(parts)==1:
        case =0
    elif len(parts) == 3:
        case = 4
    elif parts[1] == "":
        case = 1
    elif parts[0] == "":
        case = 2
    elif parts[0] != "" and parts[1] != "":
        case = 3

    #Case 4 is dealt sperately as it has 2 sub queries    
    if case == 4:
        if parts[0] == "":
            case = 1
    
    #We can see which of the above case our query belongs to using this
    #print("case =", case)

    # Read Inverted Index
    inverted = {}
    with open("index/invertedindex.txt") as f:
        for line in f:
            temp = line.split()
            val = temp[1].split(",")
            inverted[temp[0]] = val


    # Read Permuterm Index
    permuterm = {}
    with open("index/permutermindex.txt") as f:
        for line in f:
            temp = line.split()
            permuterm[temp[0]] = temp[1]
     
    #print(permuterm)

    final_result.append('This is our Permuterm Index corresponding to each word')
    final_result.append(permuterm)

    
    #This function will match the prefix of the word/wildcard query to the words in index
    def prefix_match(term, prefix):
        term_list = []
        for tk in term.keys():
            if tk.startswith(prefix):
                final_result.append('This is the Permuterm Index where wildcard query is matched')
                final_result.append(tk)
                term_list.append(term[tk])
        return term_list

    docID = 0
    
    #This function is used to process query (ie after prefix match, the word and document is extracted where the prefix match has occured)
    def process_query(query):    
        term_list = prefix_match(permuterm,query)
        #print(term_list)
        final_result.append('This is out List contating the terms which match our desired query')
        final_result.append(term_list)

        #This is the list contating document IDs having the required word in query
        docID = []
        for term in term_list:
            docID.append(inverted[term])

        temp = []
        for x in docID:
            for y in x:
                temp.append(y)     

        temp = [int(x) for x in temp]
        documentID = 0
        path = "dataset"
        for root, dirs, files in os.walk(path):
            for file in files:
                documentID = documentID + 1
                with open(os.path.join(path, file)) as f:
                    for text in f:
                        if documentID in temp:
                            #print(file + "\n" + text + "\n")
                            output_list.append(file + '\n'+  text)
                f.close()
                
        final_result.append('This is the final list containing documnets and their ID')
        final_result.append(output_list)
    
    #Queries are processed on the bases of their cases
    if case == 0:
        pass
    elif case == 1:
        query = "$" + parts[0]
        final_result.append('This is how the query will be processed')
        final_result.append(query)
    elif case == 2:
        query = parts[1] + "$"
        final_result.append('This is how the query will be processed')
        final_result.append(query)
    elif case == 3:
        query = parts[1] + "$" + parts[0]
        final_result.append('This is how the query will be processed')
        final_result.append(query)      
    elif case == 4:
        queryA = parts[2] + "$" + parts[0]
        queryB = parts[1]
        final_result.append('This is how the query will be processed')
        final_result.append([queryA, queryB])

    if case != 4:
        process_query(query)
    elif case == 4:
    # queryA Z$X
        term_list = prefix_match(permuterm,queryA)
        final_result.append('This is out List contating the terms which match our desired queryA')
        final_result.append(term_list)

        docID = []
        for term in term_list:
            docID.append(inverted[term])

        temp1 = []
        for x in docID:
            for y in x:
                temp1.append(y)     

        temp1 = [int(x) for x in temp1]
        term_list = prefix_match(permuterm,queryB)
        final_result.append('This is out List contating the terms which match our desired queryB')
        final_result.append(term_list)

        docID = []
        for term2 in term_list:
            docID.append(inverted[term2])

        temp2 = []
        for x in docID:
            for y in x:
                temp2.append(y)       

        temp2 = [int(x) for x in temp2]

        #Here bitwise function is used to find the common words from both the queries
        temp = bitwise_and(temp1,temp2)
        final_result.append('This is List contating common term documents for queryA and queryB')
        final_result.append(term)
        
        documentID = 0
        path = "dataset"
        for root, dirs, files in os.walk(path):
            for file in files:
                documentID = documentID + 1
                with open(os.path.join(path, file)) as f:
                    for text in f:
                        if documentID in temp:
                            #print(file + "\n" + text + "\n")
                            output_list.append(file + '\n'+  text)
                f.close()
                
        final_result.append('This is the final list containing documnets and their ID')
        final_result.append(output_list)
    
    
    return(output_list, final_result)

In [25]:
output_list, final_result= querying('arpan')

In [26]:
output_list

['doc1.txt\nbig data is the future arpan']

## Simply Animated

In [31]:
output_list, final_result= querying('a*')

In [32]:
#Text Animation 
def animate_text(function):
    for x in function:
        if type(x)==dict:
            for permuterm in x:
                print('This is our Permuterm Index corresponding to the word: \"' + x[permuterm] + '\"')
                
                print(permuterm)
                time.sleep(0.2)
                #os.system('clear')
                clear_output(wait=True)
        else:    
            #print("\n")
            print(x)
            time.sleep(2)
            #os.system("clear") 
            clear_output(wait=True)

        
#Main Program Starts Here....
animate_text(final_result)

['doc1.txt\nbig data is the future arpan']


## Boolean Queries

In [29]:
def boolean(query):
    #Firstly queries are splitted 
    pieces = shlex.split(query)
    include, exclude = [], []
    for piece in pieces:
        
        if piece.startswith('-'):
            exclude.append(piece[1:])
        else:
            include.append(piece)
     
     #print(include)
     #print(exclude)
     
    #Make include_set and exclude_set having documnets where query to be included and excluded respectively.
    include_set = set(querying(include[0])[0])
    exclude_set = set()
    
    for x in include[1:]:
        include_set= set(querying(x)[0]).intersection(include_set)
    
    for x in exclude:
        exclude_set= set(querying(x)[0]).union(exclude_set)
    
    #print(include_set)
    #print(exclude_set)
    
    final_set= list(include_set.difference(exclude_set))
    
    return final_set

In [30]:
boolean('d* -o*')

['doc1.txt\nbig data is the future arpan',
 'doc3.txt\nlinkedin predicts data mining and statistics bubble burst']